#**Actividad 3: Reglas de Asociación**

###Apellidos y nombres: Dorado Torres Diego Alonso
###Código:170594

### 1.-Instalar bibliotecas






In [38]:
import numpy as np
import pandas as pd


### 2.-Importar datos

In [39]:
# Leer archivo .npy
db = np.load("spotify.npy", allow_pickle = True)

### 3.-Implementar el algoritmo Apriori

####a) Función frecuencia itemsets

In [40]:
def get_frequent_itemsets(playlists, min_support):
  # Listar los datos
  playlists = list(playlists.item().values())
  # Recorrer los playlists
  playlists = [set(playlist) for playlist in playlists]
  canciones = [item for sublista in playlists for item in sublista]
  # Frecuencia de cada playlist
  frecuencia_playlists = pd.Series(data=canciones).value_counts().to_dict()
  # Diccionario de playlists
  canciones_playlists = collections.defaultdict(set)
  # Valor para cada playlist
  for index, playlist in enumerate(playlists):
      for cancion in playlist:
          canciones_playlists[cancion].add(index)
  canciones_playlists = canciones_playlists
  # Determinar las mejores canciones en cada playlist
  contador = {cancion: veces for cancion, veces in frecuencia_playlists.items() if veces / len(playlists) >= min_support}
  itemsets = [{cancion} for cancion in contador.keys()]
  # Obtener los itemsets con más frecuencia
  itemsets_frecuentes = {}
  itemsets_frecuentes[1] = sorted(contador.items(), key=lambda x: x[1], reverse=True)
  itemsets_frecuentes_tamaño = []
  # Inicializar contador
  k = 2
  actual = itemsets  
  while len(actual) != 0:
    # Generar itemsets
    combinaciones = set() 
    m = k - 2
    for candidato in actual: 
      candidato = list(candidato)
      for aux in actual:
        aux = list(aux)
        union = True
        for i in range(k - 2):
          if candidato[i] != aux[i]:
              union = False
              break
        if not union:
          continue
        if candidato[k - 2] < aux[k - 2]:
          c = candidato + [aux[k - 2]]
          c = frozenset(sorted(c))
          combinaciones.add(c)
    # Hallar que dicho itemset cumpla cumpla con el umbral de soporte mínimo
    contadorDecombinaciones = {}   
    playlist_tamanio = len(playlists)   
    for candidato in combinaciones:
        playlists_inter = []
        for song in candidato:
          playlists_inter.append(canciones_playlists[song])
        contadorDecombinaciones[candidato] = len(set.intersection(*playlists_inter))
    contador_libreria = {subset: veces for subset, veces in contadorDecombinaciones.items() if veces / playlist_tamanio >= min_support} 
    libreria = contador_libreria.keys() 
    itemsets_frecuentes_tamaño.extend(libreria)
    itemsets_frecuentes[k] = sorted(contador_libreria.items(), key=lambda x: x[1], reverse=True)
    k += 1
    actual = libreria
  # Generar itemsets con pd.DataFrame
  itemsets_dataframe = pd.DataFrame([item for sublista in itemsets_frecuentes.values() for item in sublista]).round(3)
  itemsets_dataframe.columns = ["itemset", "contador_support"]
  itemsets_dataframe["support"] = itemsets_dataframe["contador_support"] / len(playlists)
  extra_data = []
  # Itemsets más frecuentes en 1er lugar
  extra_data.append(itemsets_dataframe)
  # En 2do lugar los datos de los playlists
  extra_data.append(playlists)
  # En 3ra posición el registro de canciones
  extra_data.append(canciones_playlists)
  # En 4ta posicion el registro de los itemsets largos
  extra_data.append(itemsets_frecuentes_tamaño)
  return extra_data


####b) Función generar reglas de asociación

In [36]:
def generate_association_rules(frequent_itemsets, confidence, lift):
  reglas = [] 
  for itemset in frequent_itemsets[3]:
    #Formar todas las combinaciones de reglas de asociación
    itemset_contador = calcular_interseccion(itemset,frequent_itemsets[2])
    itemset_support = itemset_contador / len(frequent_itemsets[0])
    #Recorrer los itemsets
    for i in range(1, len(itemset) + 1):
        for xSet in combinaciones(itemset, i):
            xSet = set(xSet)
            ySet = set(itemset) - xSet
            #Hallar intersección
            xSupport = calcular_interseccion(xSet,frequent_itemsets[2]) / len(frequent_itemsets[0])
            xySupport = calcular_interseccion(xSet.union(ySet),frequent_itemsets[2]) / len(frequent_itemsets[0])
            rule_confidence = xySupport / xSupport
            if len(xSet) > 0 and len(ySet) > 0:
                ySupport = calcular_interseccion(ySet,frequent_itemsets[2]) / len(frequent_itemsets[0])
                #Hallar reglas
                rule_lift = xySupport / (xSupport * ySupport)
                reglas.append((xSet, ySet, rule_confidence, xySupport, rule_lift))
  #Generar pd.dataframe de las reglas de asociacion
  reglas_dataframe = pd.DataFrame(data=reglas, columns=["X", "Y", "confidence", "support", "lift"]).round(3)
  reglas_dataframe["X"] = list(map(tuple, reglas_dataframe["X"]))
  reglas_dataframe["Y"] = list(map(tuple, reglas_dataframe["Y"]))
  #Hallar las reglas de asociacion de 10 playlists para la confianza
  order_by="confidence"
  n=10
  confidence_df = reglas_dataframe.sort_values(order_by, ascending=False).head(n)
  #Hallar las mejores reglas de asociacion
  order_by="lift"
  n=10
  lift_df2 = reglas_dataframe.sort_values(order_by, ascending=False).head(n)
  #Hallar las 10 mejores reglas de asociacion que cumplan con ambas condiciones
  reglas_dataframe["len_itemset"] = reglas_dataframe.apply(lambda x: len(set(x["X"]).union(set(x["Y"]))), axis=1)
  reglas_dataframe=reglas_dataframe.sort_values(by="len_itemset", ascending=False)
  both_rules = reglas_dataframe[(reglas_dataframe["confidence"] >= confidence) & (reglas_dataframe["lift"] >= lift)]

  return confidence_df,lift_df2,both_rules

### 4.- Aplicar el algoritmo y obtener reglas de asociación

In [41]:
# Generar itemsets frecuentes
itemsets = get_frequent_itemsets(db,0.01)
itemsets[0].sort_values(by="support").head()
# Generar reglas de asociación
reglas_confidence,reglas_lift,reglas_ambos = generate_association_rules(itemsets,0.4,0.5)
# Mostrar reglas
reglas_confidence
reglas_lift
reglas_ambos.head(10)
# Seleccionar las mejores 4 reglas
mejores=reglas_lift.head(4)
mejores

,X,Y,confidence,support,lift
161,"(No Heart,)","(X (feat. Future),)",0.696,0.101,3.493
160,"(X (feat. Future),)","(No Heart,)",0.505,0.101,3.493
24,"(Knee Deep (feat. Jimmy Buffett),)","(Chicken Fried,)",0.682,0.104,3.261
25,"(Chicken Fried,)","(Knee Deep (feat. Jimmy Buffett),)",0.500,0.104,3.261


### 5.- Explicar las reglas obtenidas

Se determino las 4 reglas de acuerdo a los diferentes indicadores disponibles con soporte, confianza y lift,ya que se pueden encontrar coincidencias con las búsquedas relacionada. 

De acuerdo a lo establecido según el género y/o el artista de los playlists se pueden hallar distintas reglas entonces el umbral será mayor al incial con más reglas de acuerdo al grado de confianza y soporte.


###Primera regla 
De acuerdo a los resultados se observa que los playlist son del mismo género, entonces esto se agrega a las reglas de asociación y aumenta el umbral obtenido.

###Segunda regla 
En esta regla se observa lo establecido ya que se repite los mismos playlists con la anterior regla de asociación por lo tanto existe una mejor confianza en el umbral.

###Tercera regla
En esta regla también se observa lo establecido ya que en esta regla se debe a las preferencias por lo cual el umbral es más alto..

###Cuarta regla 
En esta regla se observa y se cumple lo establecido lo cual hace que el umbral sea mayor de acuerdo a la confianza y el soporte.

De acuerdo a lo establecido según el género y/o el artista de los playlists se pueden hallar distintas reglas entonces el umbral será mayor al incial con más reglas de acuerdo al grado de confianza y soporte.

